# Laboratorio #4: Procesamiento del Lenguaje Natural

El procesamiento del lenguaje natural, también conocido como NLP (por sus siglas en inglés), es un subcampo de la inteligencia artificial que permite a las computadoras comprender el lenguaje humano. 

EL NLP está a nuestro alrededor: extracción de información, análisis de sentimientos, traducción automática (por ejemplo, el traductor de Google), detección de spam, autocompletar y bots de chat, por nombrar algunos.

El lenguaje humano está lleno de ambigüedades, las cuales requerien de análisis complejos para que una computadora pueda entender el significado real de la frase. Por ejemplo, en la oración:
<div style="text-align:center;">
    <p>La mujer golpeó a un hombre con un paraguas.</p>
</div>
puede tener dos significados distintos: la mujer golpeó a un hombre el cual tenía un paraguas o, la mujer golpeó con su paraguas a un hombre. Existen algoritmos y técnicas para desambiguar las frases.

Muchos de los SRI actuales ofrecen la posibilidad a los usuarios de autocompletar las freses que este introduce y esto lo hacen a través de modelos de lenguajes y representaciones "cómodas" que le permiten predecir el texto del usuario. 

El objetivo de la clase será **implementar un conjunto de funciones que permitan predecir texto**, de forma similar a la mostrada en la figura siguiente: 
<div style="text-align:center;">
    <img src="predecir-texto.png" width="250">
</div>

Comencemos importando las bibliotecas necesarias.

In [1]:

# Permitirá el preprocesamiento con los textos
import spacy
nlp = spacy.load("en_core_web_sm")

# Función para crear los n-gramas
from nltk import ngrams

import nltk

# Almacenará las relaciones entre la información representada con los n-gramas
from nltk.probability import ConditionalFreqDist

# Funciones auxiliares útiles 
from teacher_help import get_text

### Ejercicio #1: Implemente una función para procesar textos.

Para la resolución del ejercicio puede considerar:
- Mantener solo el texto, de la misma forma en que lo encuentre.
- No preservar números, signos de puntuación u otros términos con caracteres esepciales.

###### Pista: Ya viste algo similar a la respuesta esperada. Intenta recordar.

In [2]:
def normalize_text(text):
    """Normalizes the text to be processed

    Args:
        text (str): Text to process.
        
    Return:
        [str]: Text to process
    
    """
    # raise NotImplementedError('Ejercicio #1')
    doc = nlp(text)
    return [token.text.lower() for token in doc if token.lemma_.isalpha()]

In [3]:
text = '''
The tree is empty. 
Tomorrow will be a nice day. 
The tree is blue and has dreams
'''
words = normalize_text(text)

print('Texto normalizado:', words)

Texto normalizado: ['the', 'tree', 'is', 'empty', 'tomorrow', 'will', 'be', 'a', 'nice', 'day', 'the', 'tree', 'is', 'blue', 'and', 'has', 'dreams']


---

El próximo paso es establecer una relación entre los términos del texto almacenado para la predicción. Nótese que para esta tarea, se necesita una base de datos previa, puesto que el sistema no puede predecir el próximo término de una frase si no conoce de antemano posibles frases.

Luego, a través de un ejemplo simple puede demostrarse la dependencia entre algunas palabras, por ejemplo: es más probable que aparezca la frase "El gato está durmiendo" que "El gato es bailando". Si esta idea se lleva al campo de las probabilidades puede denotarse como:
$$P(\text{``El gato está durmiendo''})>P(\text{``El gato está bailando''})$$ 
donde $P$ denota la probabilidad de que la oración.

Esta idea puede aplicarse usando el modelo de lenguaje de N-Gramas, el cual predice la probabilidad de la ocurrencia de una secuencia de $N$ términos. Recuerde que la representación BoW (bolsa de palabras) es un modelo donde $N=1$.

Para que el modelo N-Grama funcione correctamente se usará la suposición de Markóv, la cual plantea que la probabilidad de que aparezca un término solo dependerá de sus $N-1$ términos anteriores.

### Ejercicio #2: Contruya un modelo N-Grama para la representación de la información y sus relaciones.

Considere las siguientes ideas durante la resolución de la función:

- Se trabajará con n-gramas de tamaño 3.

- La función `ngrams` construirá los n-gramas. Para su ejecución, además de la lista de términos, puede hacer uso de otros parámetros como:
  - pad_left (boolean): Indica si el primer n-grama detectado será: `('unknown', 'unknown', 'el')` o `('el', 'gato', 'está')`.
  - pad_right=(boolean): Indica si el último n-grama detectado será: `('gato', 'está', 'durmiendo')` o `('durmiendo', 'unknown', 'unknown')`.
  - left_pad_symbol (str): Si `pad_left=True` define el caracter a poner por 'unknown'.
  - right_pad_symbol (str): Si `pad_right=True` define el caracter a poner por 'unknown'.

- La clase `ConditionalFreqDist` permite crear un diccionario con cierta particularidad. Un diccionario predeterminado es la estructura de datos subyacente en esta clase y cuenta la frecuencia de co-ocurrencia y pares clave-valor. Almacena los recuentos de una palabra (w3) después de las 2 palabras anteriores (w1, w2) en el trigrama. Si el recuento de w3 no está presente, se utiliza un valor predeterminado. Por ejemplo:
  ```python
  tmp = ConditionalFreqDist()
  tmp[('a', 'b')]['c'] = 4 
  tmp[('a', 'b')]['d'] = 2 
  tmp[('c', 'd')]['a'] = 1 
  total = float(sum(tmp[('a', 'b')].values()))
  tmp[('a', 'b')]['c'] /= total

  from pprint import pprint
  pprint(list(tmp.items()))
  ```

- Normalice los valores, de forma que lo mantenido en cada elemento de la relación es una probabilidad.

In [4]:
def n_gram_model(text):
    """Build the N-Gram model

    Args:
        text ([str]): List of pre-processed terms.

    Returns:
        nltk.probability.ConditionalFreqDist: Relationships between n-grams according to the normalized frequency of occurrence
    """
    # raise NotImplementedError('Ejercicio #2')
    
    trigrams = list(ngrams(text, 3, pad_left=True, pad_right=True, 
                           left_pad_symbol='<s>', 
                           right_pad_symbol='</s>'))
    
    # # Image
    # freq_tri = nltk.FreqDist(trigrams)
    # freq_tri.plot(30, cumulative=False)
    # print("Most common trigrams: ", freq_tri.most_common(5))
    
    # make conditional frequencies dictionary
    cfdist = ConditionalFreqDist()
    for w1, w2, w3 in trigrams:
        cfdist[(w1, w2)][w3] += 1

    # transform frequencies to probabilities
    for w1_w2 in cfdist:
        total_count = float(sum(cfdist[w1_w2].values()))
        for w3 in cfdist[w1_w2]:
            cfdist[w1_w2][w3] /= total_count

    return cfdist

In [5]:
model = n_gram_model(words)

from pprint import pprint
print('Información del modelo:')
pprint(list(model.items()))

Información del modelo:
[(('<s>', '<s>'), FreqDist({'the': 1.0})),
 (('<s>', 'the'), FreqDist({'tree': 1.0})),
 (('the', 'tree'), FreqDist({'is': 1.0})),
 (('tree', 'is'), FreqDist({'empty': 0.5, 'blue': 0.5})),
 (('is', 'empty'), FreqDist({'tomorrow': 1.0})),
 (('empty', 'tomorrow'), FreqDist({'will': 1.0})),
 (('tomorrow', 'will'), FreqDist({'be': 1.0})),
 (('will', 'be'), FreqDist({'a': 1.0})),
 (('be', 'a'), FreqDist({'nice': 1.0})),
 (('a', 'nice'), FreqDist({'day': 1.0})),
 (('nice', 'day'), FreqDist({'the': 1.0})),
 (('day', 'the'), FreqDist({'tree': 1.0})),
 (('is', 'blue'), FreqDist({'and': 1.0})),
 (('blue', 'and'), FreqDist({'has': 1.0})),
 (('and', 'has'), FreqDist({'dreams': 1.0})),
 (('has', 'dreams'), FreqDist({'</s>': 1.0})),
 (('dreams', '</s>'), FreqDist({'</s>': 1.0}))]


---

Una vez hecho el modelo, ya es posible predecir frases.

### Ejercicio #3: Implemente una función que dado un modelo n-grama y una frase, devuelva una lista del posible término siguiente.


In [6]:
def predict(model, phrase):
    """Predict the next term in a sentence

    Args:
        model (ConditionalFreqDist): Model.
        phrase (_type_): Phrase.

    Returns:
        [str]: List of possible terms, ordered from highest to lowest probability of appearance.
    
    """
    # raise NotImplementedError('Ejercicio #3')
    
    phrase = normalize_text(phrase)

    w1 = len(phrase) - 2
    w2 = len(phrase)
    prev_words = phrase[w1:w2]

    # display prediction from highest to lowest maximum likelihood
    return sorted(dict(model[prev_words[0], prev_words[1]]), 
    key=lambda x: dict(model[prev_words[0], prev_words[1]])[x], 
    reverse=True)
    

In [7]:
phrase = 'tree is'
predictions = predict(model, phrase)
print("Predicción del modelo trigrama: ", predictions)

Predicción del modelo trigrama:  ['empty', 'blue']


### ¿Terminaste? ¡Yeah!

Prueba ahora con un ejemplo real. Los textos son libros famosos. Ejecuta la siguiente celda y define tu frase a completar.

In [8]:
print('Normalizando los textos ...')
words = get_text('./books', normalize_text)

print('Creando el modelo ...')
model = n_gram_model(words)

Normalizando los textos ...
Creando el modelo ...


In [9]:
phrase = 'Alice had no very'
predictions = predict(model, phrase)
print("Trigram model predictions: ", predictions)

Trigram model predictions:  ['great', 'cordial', 'striking', 'difficult', 'good', 'sweet', 'clear']
